# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
input_data_file = "../output_data/cities.csv"

#Read WeatherPy CSV file
Final_city_data= pd.read_csv(input_data_file)

Final_city_data.head()

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Qaqortoq,60.7167,-46.0333,41.00,36,75,26.46,GL,1610414487
1,Ponta do Sol,32.6667,-17.1000,55.00,66,9,3.00,PT,1610414126
2,Bluff,-46.6000,168.3333,63.25,57,13,2.10,NZ,1610414251
3,Carnarvon,-24.8667,113.6333,80.60,74,75,21.85,AU,1610414250
4,Albany,42.6001,-73.9662,28.99,67,18,2.71,US,1610414180


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = Final_city_data[["Latitude", "Longitude"]].astype(float)

humidity= Final_city_data["Humidity"].astype(float)


In [4]:
fig= gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig = gmaps.figure(center=(0, 0), zoom_level=2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the cities to fit weather conditions.
Perfect_humidity= Final_city_data.loc[Final_city_data["Humidity"] <= 68]

Perfect_temp= Perfect_humidity.loc[Perfect_humidity["Max Temperature"] > 75 ]

#Store into variable named hotel_df.
hotel_df= Perfect_temp.loc[Perfect_temp["Cloudiness"] <= 30 ]

hotel_df.head()

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
24,Hilo,19.7297,-155.0900,80.60,61,1,11.50,US,1610414274
73,Waingapu,-9.6567,120.2641,85.64,64,3,3.13,ID,1610414499
99,Maningrida,-12.0667,134.2667,89.58,63,5,2.93,AU,1610414297
110,Mount Isa,-20.7333,139.5000,93.20,36,0,6.91,AU,1610414328
201,Merauke,-8.4667,140.3333,85.57,67,26,1.59,ID,1610414316


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df.head()

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
24,Hilo,19.7297,-155.0900,80.60,61,1,11.50,US,1610414274,
73,Waingapu,-9.6567,120.2641,85.64,64,3,3.13,ID,1610414499,
99,Maningrida,-12.0667,134.2667,89.58,63,5,2.93,AU,1610414297,
110,Mount Isa,-20.7333,139.5000,93.20,36,0,6.91,AU,1610414328,
201,Merauke,-8.4667,140.3333,85.57,67,26,1.59,ID,1610414316,


In [7]:
# Hit the Google Places API for each city's coordinates.
target_search = "Hotel"
target_radius = 5000


#Set parameters to search for specifics
params = { 
    "keyword": target_search,
    "radius": target_radius, 
    "key":g_key, 
}

In [8]:
print("Beginning Data Retrieval")
print("-------------------------")

# Use iterrows to iterate through dataframe
for index, row in hotel_df.iterrows():

    #Store latitude and longitude from hotel_df
    lat = row['Latitude']
    lng = row['Longitude']


    #Update location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"


    #Set the base_url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_names = requests.get(base_url, params=params)
    hotel_names = hotel_names.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_names["results"][0]["name"]

    except (KeyError, IndexError, TypeError):
        print("Missing result. Skipping... .")

Beginning Data Retrieval
-------------------------
Missing result. Skipping... .
Missing result. Skipping... .
Missing result. Skipping... .
Missing result. Skipping... .
Missing result. Skipping... .


In [9]:
pprint(hotel_names)

NFsfSRxsP8ElGLXNmTYGzyY',
                          'width': 800}],
              'place_id': 'ChIJ65JsYnsGw2oRV0JAcg8Zses',
              'plus_code': {'compound_code': 'R48V+4F Mildura, Victoria, '
                                             'Australia',
                            'global_code': '4RQ4R48V+4F'},
              'rating': 4.3,
              'reference': 'ChIJ65JsYnsGw2oRV0JAcg8Zses',
              'scope': 'GOOGLE',
              'types': ['real_estate_agency',
                        'travel_agency',
                        'lodging',
                        'point_of_interest',
                        'establishment'],
              'user_ratings_total': 12,
              'vicinity': '202 Ontario Ave, Mildura'},
             {'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -34.1947471, 'lng': 142.151578},
                           'viewport': {'northeast': {'lat': -34.19331552010728,
                                                  

In [10]:
# Save Data to csv file
hotel_df.to_csv("../output_data/Perfect_weather_hotels.csv", index= False, header= True)

# Read CSV 
Perfect_weather_hotels= pd.read_csv("../output_data/Perfect_weather_hotels.csv")

Perfect_weather_hotels.head()

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Hilo,19.7297,-155.0900,80.60,61,1,11.50,US,1610414274,Hilo Hawaiian Hotel
1,Waingapu,-9.6567,120.2641,85.64,64,3,3.13,ID,1610414499,NaN
2,Maningrida,-12.0667,134.2667,89.58,63,5,2.93,AU,1610414297,Dhukurrdji Lodge
3,Mount Isa,-20.7333,139.5000,93.20,36,0,6.91,AU,1610414328,ibis Styles Mt Isa Verona
4,Merauke,-8.4667,140.3333,85.57,67,26,1.59,ID,1610414316,NaN


In [11]:
Renamed_df = Perfect_weather_hotels.rename(columns= {"City Name" : "City"})

Renamed_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Hilo,19.7297,-155.0900,80.60,61,1,11.50,US,1610414274,Hilo Hawaiian Hotel
1,Waingapu,-9.6567,120.2641,85.64,64,3,3.13,ID,1610414499,NaN
2,Maningrida,-12.0667,134.2667,89.58,63,5,2.93,AU,1610414297,Dhukurrdji Lodge
3,Mount Isa,-20.7333,139.5000,93.20,36,0,6.91,AU,1610414328,ibis Styles Mt Isa Verona
4,Merauke,-8.4667,140.3333,85.57,67,26,1.59,ID,1610414316,NaN
5,Port Macquarie,-31.4333,152.9167,86.00,41,0,1.99,AU,1610414176,Ana Mandara Luxury Bed & Breakfast
6,Karratha,-20.7377,116.8463,95.25,41,0,14.38,AU,1610414289,NaN
7,Lafiagi,8.8667,5.4167,76.35,23,0,0.83,NG,1610414833,One to Z Hotel
8,Murchison,-41.8000,172.3333,79.79,29,0,3.09,NZ,1610414850,Murchison Lodge
9,Bima,-8.4667,118.7167,87.67,67,2,0.78,ID,1610414303,Hotel Marina


Plot markers on top of the heatmap.

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Renamed_df.iterrows()]
locations = Renamed_df[["Latitude", "Longitude"]]


In [13]:
# Add marker layer 

marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig= gmaps.figure()

#Zoom in before adding layers
fig = gmaps.figure(center=(0, 0), zoom_level=2)

#Add marker layer
fig.add_layer(marker_layer)

#Add heat layer
fig.add_layer(heat_layer)

# fig = gmaps.figure(center=(0, 0), zoom_level=2)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))